In [38]:
# -*- coding: utf-8 -*-
import sys
import pandas
import time
import datetime

from selenium import webdriver

#required for wait untill
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains

## Fetch Data

In [46]:
#location of excel
path = "D:/User/Desktop/Insta/Hashtags.xlsx"

df = pandas.read_excel(path,sheet_name=None)

# from the Hashtags sheet, fetch the Hashtag Column
hashtags = df['Hashtags']['Hashtag'].tolist()


In [40]:

def GetHashtagInformation(browser):
    '''fetches info on the current page'''
    
    try:
        #wait for page to load
        test = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/header/div[2]/div[1]/div[2]/span/span')

        #fetch number of posts
        pubnumber = int(test.text.replace(" ",""))
        
    except:
        print("number of publications not found or # does not exist")
        pubnumber = -1



    # get average likes and comments on top 9 posts
    try:

        # fetch the class of a post, facebook is sneaky this changes sometimes, this way it always finds the correct class
        classtosearch = browser.find_element_by_xpath('//h2/following-sibling::div/div/div/div').get_attribute('class')
        
        # find all posts
        test= browser.find_elements_by_xpath("//*[@class='"+classtosearch+"']")
  
        likes=0
        comments=0

        # if less than 9, set it as the max
        maxcount = 9
        if len(test)<maxcount:
            maxcount = len(test)
        

        for i in range(maxcount):
            
            # fetch it
            lik, com = GetLikesandComments(test[i],browser)


            # increase counts
            likes = likes + lik
            comments = comments + com

        # average them
        comments = round(comments / maxcount,2)
        likes = round(likes / maxcount,2) 

    except Exception as e:  
        print("comments and likes not found")
        likes=0
        comments=0

    #publications, avglikes , avgcomments
    
    return [pubnumber,likes,comments]

def GetLikesandComments(post,browser):
    '''fetch likes and comments for a specific post'''
    
    #mouse over post, so that likes and comments show
    hover = ActionChains(browser).move_to_element(post)
    hover.perform()
    
    #sketit
    elements = browser.find_elements_by_xpath('//li/span[1]')
    likes=elements[0].text
    comments=elements[1].text
    
    #likes , comments
    try:
        # convert to numbers
        return instastring2number(likes) ,  instastring2number(comments)
    except:
        print("Error Ocurred")
        return -1,-1

def instastring2number(s):
    '''Converts instagram string into a number'''
    
    multiplier=1
    if 'k' in s:
        s = s.replace('k','')
        multiplier=1000
    elif 'm' in s:
        s = s.replace('m','')
        multiplier=1000000
    
    
    s = s.replace(',','.')
    s = s.replace(' ','')
            
    return float(s)*multiplier

In [41]:
#initialize selenium thingy
browser = webdriver.Chrome()
#has to be chrome because Firefox does not auto scroll, with move_to_element, it would have to be implemented

In [42]:
#default url where hashtags will be appended
baseurl = "https://www.instagram.com/explore/tags/"

entries=[]

#cycles all hashtags

count = 0

for hashtag in hashtags:
    
    count=count+1
    
    browser.get(baseurl+hashtag)
    
    #keeping count
    print(str(count)+"/"+str(len(hashtags))+"\t"+hashtag)


    #wait to see if pageloads/hashtagexists
    time.sleep(2)

    #fetch  Publications","DailyPosts","AvgLikes
    result = GetHashtagInformation(browser)   
    
    # creates tuple and that is ("Hashtags","Publications","DailyPosts","AvgLikes") and adds it a list
    entries.append(tuple([hashtag] + result))
    
    # show result
    print(entries[-1])

    
browser.quit()






1/252	planearparanaochorar
comments and likes not found
('planearparanaochorar', 1, 0, 0)
2/252	prepararumaentrevista
comments and likes not found
('prepararumaentrevista', 1, 0, 0)
3/252	fazercontasavida
19.0 6.0
13.0 0.0
5.0 0.0
6.0 0.0
11.0 2.0
5.0 7.0
30.0 1.0
67.0 0.0
('fazercontasavida', 8, 19.5, 2.0)
4/252	subcontratar
105.0 6.0
9.0 1.0
35.0 0.0
59.0 9.0
9.0 0.0
13.0 0.0
4.0 0.0
48.0 6.0
13.0 3.0
('subcontratar', 146, 32.78, 2.78)
5/252	guiafiscal
80.0 2.0
10.0 0.0
57.0 2.0
17.0 0.0
87.0 13.0
60.0 0.0
28.0 0.0
10.0 0.0
6.0 0.0
('guiafiscal', 12, 39.44, 1.89)
6/252	trabalhadorporcontadeoutrem
70.0 9.0
42.0 6.0
73.0 5.0
('trabalhadorporcontadeoutrem', 3, 61.67, 6.67)
7/252	japoupastehoje
347.0 3.0
49.0 6.0
53.0 2.0
49.0 5.0
19.0 3.0
44.0 1.0
22.0 2.0
32.0 2.0
35.0 5.0
('japoupastehoje', 339, 72.22, 3.22)
8/252	sobraxis
63.0 20.0
100.0 4.0
238.0 26.0
209.0 6.0
81.0 22.0
57.0 8.0
comments and likes not found
('sobraxis', 119, 0, 0)
9/252	saberempreender
192.0 2.0
259.0 1.0
41.0 1.0


528.0 18.0
909.0 14.0
312.0 28.0
18400.0 80.0
108.0 15.0
485.0 14.0
505.0 7.0
124.0 59.0
('escritacriativa', 95343, 2431.33, 27.11)
56/252	financasemdia
1260.0 32.0
457.0 52.0
152.0 9.0
803.0 13.0
169.0 10.0
605.0 48.0
1246.0 25.0
124.0 24.0
755.0 21.0
('financasemdia', 7646, 619.0, 26.0)
57/252	remuneração
434.0 19.0
274.0 51.0
753.0 35.0
118.0 9.0
1269.0 32.0
661.0 11.0
107.0 5.0
547.0 20.0
102.0 2.0
('remuneração', 6809, 473.89, 20.44)
58/252	chegadedesculpas
8951.0 249.0
8352.0 183.0
4744.0 63.0
5092.0 55.0
583.0 18.0
166.0 21.0
594.0 24.0
738.0 19.0
2142.0 97.0
('chegadedesculpas', 39634, 3484.67, 81.0)
59/252	vidadeadulto
961.0 25.0
1476.0 32.0
1106.0 55.0
493.0 78.0
1352.0 76.0
1517.0 83.0
719.0 54.0
627.0 31.0
599.0 96.0
('vidadeadulto', 15551, 983.33, 58.89)
60/252	empreendedorismoeinovação
153.0 28.0
342.0 11.0
112.0 6.0
1548.0 44.0
162.0 10.0
264.0 39.0
1273.0 6.0
256.0 23.0
118.0 15.0
('empreendedorismoeinovação', 6563, 469.78, 20.22)
61/252	negocioglobal
3077.0 98.0
118.0 

731.0 5.0
513.0 5.0
2706.0 20.0
480.0 4.0
577.0 8.0
1554.0 8.0
577.0 10.0
('mentesdesucesso', 27190, 1153.0, 12.33)
105/252	inovacao
9092.0 35.0
20100.0 287.0
3061.0 348.0
4576.0 66.0
2311.0 18.0
7776.0 65.0
431.0 17.0
988.0 28.0
9243.0 155.0
('inovacao', 708005, 6397.56, 113.22)
106/252	emprendedorismo
714.0 17.0
2813.0 27.0
118.0 17.0
1297.0 26.0
132.0 2.0
2345.0 31.0
2695.0 34.0
5024.0 391.0
2607.0 14.0
('emprendedorismo', 134662, 1971.67, 62.11)
107/252	dinheironainternet
2875.0 42.0
1808.0 16.0
5254.0 144.0
1151.0 39.0
2101.0 46.0
1865.0 14.0
1485.0 95.0
2322.0 95.0
1048.0 10.0
('dinheironainternet', 89214, 2212.11, 55.67)
108/252	pouparmais
7951.0 114.0
4656.0 86.0
781.0 28.0
676.0 13.0
622.0 42.0
177.0 28.0
572.0 31.0
285.0 23.0
159.0 31.0
('pouparmais', 3745, 1764.33, 44.0)
109/252	prosperidadefinanceira
3141.0 110.0
498.0 41.0
307.0 30.0
888.0 75.0
8379.0 68.0
2688.0 37.0
1490.0 32.0
1178.0 25.0
12000.0 35.0
('prosperidadefinanceira', 66381, 3396.56, 50.33)
110/252	orcamento
6

154/252	empreendedorismosocial
3028.0 57.0
2505.0 38.0
1296.0 55.0
2966.0 84.0
2996.0 40.0
2234.0 41.0
9712.0 189.0
4099.0 47.0
491.0 11.0
('empreendedorismosocial', 234761, 3258.56, 62.44)
155/252	poupar
4023.0 145.0
519.0 7.0
746.0 13.0
4028.0 72.0
3100.0 84.0
2140.0 19.0
705.0 22.0
4271.0 94.0
2100.0 57.0
('poupar', 127760, 2403.56, 57.0)
156/252	empresario
149.0 3.0
180.0 12.0
1702.0 10.0
1068.0 17.0
14800.0 291.0
4129.0 18.0
5436.0 22.0
1229.0 19.0
4841.0 60.0
('empresario', 1620365, 3726.0, 50.22)
157/252	investimento
7729.0 79.0
705.0 22.0
1338.0 23.0
2279.0 41.0
9172.0 89.0
588.0 16.0
1010.0 65.0
7822.0 362.0
12400.0 109.0
('investimento', 1882307, 4782.56, 89.56)
158/252	ganhedinheiroonline
2689.0 27.0
4491.0 50.0
4260.0 27.0
7839.0 59.0
113.0 1.0
5264.0 46.0
1319.0 51.0
10300.0 408.0
4207.0 317.0
('ganhedinheiroonline', 13467, 4498.0, 109.56)
159/252	finançaspessoais
769.0 13.0
706.0 22.0
4028.0 72.0
6361.0 75.0
511.0 11.0
1475.0 9.0
7731.0 79.0
2279.0 41.0
14200.0 100.0
('fi

202/252	dinheironaconta
594.0 14.0
1685.0 14.0
1094.0 13.0
1699.0 46.0
1151.0 39.0
1865.0 14.0
4811.0 36.0
4131.0 48.0
3957.0 81.0
('dinheironaconta', 32950, 2331.89, 33.89)
203/252	money
1557.0 122.0
1277.0 86.0
2227.0 113.0
187.0 9.0
328.0 11.0
2762.0 52.0
191.0 71.0
130.0 16.0
233.0 25.0
('money', 48002497, 988.0, 56.11)
204/252	educaçao
4365.0 90.0
5761.0 50.0
485.0 46.0
837.0 31.0
112.0 30.0
187.0 4.0
171.0 25.0
4575.0 69.0
297.0 15.0
('educaçao', 94176, 1865.56, 40.0)
205/252	portugal
1203.0 20.0
783.0 20.0
268.0 45.0
687.0 12.0
1155.0 22.0
1294.0 23.0
503.0 4.0
188.0 0.0
427.0 25.0
('portugal', 34028680, 723.11, 19.0)
206/252	reforma
778.0 35.0
1181.0 20.0
3340.0 38.0
977.0 9.0
358.0 9.0
31000.0 46.0
884.0 38.0
1262.0 34.0
521.0 12.0
('reforma', 3126608, 4477.89, 26.78)
207/252	poupanca
605.0 25.0
393.0 28.0
523.0 175.0
224.0 56.0
520.0 9.0
529.0 24.0
948.0 61.0
701.0 33.0
1178.0 17.0
('poupanca', 19633, 624.56, 47.56)
208/252	orçamento
878.0 14.0
14200.0 133.0
1856.0 41.0
1044.

250/252	negócios
8061.0 60.0
4619.0 121.0
22200.0 74.0
16500.0 234.0
8988.0 153.0
12400.0 72.0
29900.0 242.0
40600.0 2316.0
1420.0 30.0
('negócios', 1076759, 16076.44, 366.89)
251/252	dívidas
1435.0 31.0
80200.0 332.0
446.0 13.0
1019.0 95.0
81300.0 487.0
8169.0 87.0
370.0 11.0
2299.0 39.0
3258.0 82.0
('dívidas', 28446, 19832.89, 130.78)
252/252	fortuna
309.0 12.0
446.0 28.0
340.0 5.0
176.0 4.0
3654.0 63.0
2388.0 20.0
1152.0 19.0
17300.0 93.0
6770.0 188.0
('fortuna', 717889, 3615.0, 48.0)


In [53]:
# Save new data to excel (Attention this currently deletes previous data)


writer = pandas.ExcelWriter(path, engine='xlsxwriter')

#copy paste the same data back into the excel (Find a way to append a sheet for efficiency!)
for key in df:
    df[key].to_excel(writer,index=False,sheet_name=key)

#set new sheet name
sheet_name = datetime.datetime.now().strftime("%m-%d-%Y %H-%M-%S")

#set data to go in
df_info = pandas.DataFrame(entries,columns=['Hashtag','# Posts','Avg Likes','Avg Comments'])

#append it to excel
df_info.to_excel(writer,index=False,sheet_name=sheet_name)
